In [66]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import time
import os

In [56]:
def wait_with_non_gaussian_distribution():
    mean_wait_time = 8.0  # Mean wait time in seconds
    scale_parameter = 3.0  # Scale parameter for the exponential distribution
    
    wait_time = np.random.exponential(scale=scale_parameter) + mean_wait_time
    
    time.sleep(wait_time)

In [64]:


def get_artist_page(artist_name):
    base_url = 'https://www.azlyrics.com/'
    first_letter = artist_name[0].lower()
    artist_url = f"""{base_url}{first_letter}/{artist_name.replace(' ', '').lower()}.html"""

    response = requests.get(artist_url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to fetch artist page. Status code: {response.status_code}")
        return None

def scrape_lyrics(artist_name):
    lyrics_dict = {}
    artist_page = get_artist_page(artist_name)
    artist_name = artist_name.replace(' ', '').lower()
    if artist_page:
        soup = BeautifulSoup(artist_page, 'html.parser')

        album_items = soup.find_all('div', class_='listalbum-item')

        lyrics = ''
        for album_item in album_items:
            if len(album_item.find_all('a', href=True)) > 0:
                song_link = album_item.find_all('a', href=True)[0]
                song_name = re.findall(r'/(?P<content>[^/]+)\.html', song_link['href'])[0]
                song_url = f'https://www.azlyrics.com{song_link["href"]}'
                wait_with_non_gaussian_distribution()
                song_page = requests.get(song_url).text
                song_soup = BeautifulSoup(song_page, 'html.parser')
                
                lyrics_element = song_soup.find('div', class_='container main-page')

                if lyrics_element:
                    lyrics += lyrics_element.get_text(separator='\n') + '\n'
                    
                lyrics = cleaned_text = re.sub('\n+', '\n', lyrics)
            
                lyrics_dict[song_name] = lyrics

            
        return lyrics_dict
    else:
        return None

artist_name = 'men I trust'
lyrics = scrape_lyrics(artist_name)



In [65]:
lyrics.keys()

dict_keys(['acycle', 'dazed', 'staytrue', 'aprayer', 'system', 'endlessstrive', 'extaticmemoirs', 'introit', 'aclosingword', 'morsecode', 'breakforlovers', 'outinmyself', 'again', 'quiet', 'curiousfish', 'offertorio', 'plainview', 'hummingman', 'lauren', 'aquarelle', 'onclejazz', 'nortoncommanderalbumv', 'daysgoby', 'tailwhipalbumv', 'foundme', 'numbalbumv', 'saycanyouhearalbumv', 'allnight', 'ihopetobearoundalbumv', 'dorian', 'pines', 'fierogt', 'sevenalbumv', 'showmehowalbumv', 'alright', 'youdeservethisalbumv', 'pierre', 'air', 'porcelain', 'somethinginwater', 'organon', 'ohdove', 'sugar', 'sorbitol', 'treeamongshrubs', 'serenadeofwater', '5amwaltz', 'alwayslone', 'antemeridiem', 'lifelongsong', 'shoulders', 'blackholeera', 'billietoppy', 'girl', 'hardtoletgo', 'ihopetobearound', 'luckysue', 'nortoncommanderallweneed', 'numb', 'ringofpast', 'saycanyouhear', 'seven', 'showmehow', 'tailwhip', 'tides', 'youdeservethis'])

In [67]:
def write_songs_to_files(song_dict, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for song_name, song_lyrics in song_dict.items():
        file_path = os.path.join(output_directory, f"{song_name}.txt")

        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(song_lyrics)

In [68]:
write_songs_to_files(lyrics, f"lyrics_data/{artist_name.replace(' ',  '_').lower()}")